# Data Engineer Challenge

Este notebook documenta la solución completa al desafío, incluyendo la organización del repositorio, los enfoques de optimización en tiempo y memoria, el benchmarking de los backends y un anexo de verificación de resultados.

---

En primer lugar, se debe descargar el archivo `farmers-protest-tweets-2021-2-4.json` y almacenarlo en una carpeta llamada `data` en la raiz. Esto puede hacerse manualmente o ejecutando el siguiente bloque de código.

In [1]:
from pathlib import Path
import subprocess
import sys
import zipfile

data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)

json_path = data_dir / "farmers-protest-tweets-2021-2-4.json"
zip_path = data_dir / "farmers-protest-tweets-2021-2-4.json.zip"

if not json_path.exists():
    if not zip_path.exists():
        try:
            import gdown
        except ImportError:
            subprocess.run([sys.executable, "-m", "pip", "install", "gdown"], check=True)
            import gdown

        url = "https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"
        print(f"Descargando ZIP a {zip_path} ...")
        gdown.download(url, str(zip_path), quiet=False)

    print(f"Descomprimiendo {zip_path} ...")
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(path=data_dir)

    zip_path.unlink()

    print(f"Archivo JSON disponible en: {json_path}")
else:
    print(f"El archivo JSON ya existe en: {json_path}")

El archivo JSON ya existe en: ../data/farmers-protest-tweets-2021-2-4.json


## 1. Estructura del repositorio

La estructura principal del repositorio es la siguiente:

```
| .
| ├── data/
| │   └── farmers-protest-tweets-2021-2-4.json   <-- Archivo del desafío
| ├── src/
| |   ├── challenge.ipynb                  <-- Notebook principal con la solución al desafío
| │   ├── backend/
| │   │   ├── pandas_backend.py            <-- Funciones utilizando Pandas
| │   │   ├── polars_backend.py            <-- Funciones utilizando Polars
| │   │   ├── duckdb_backend.py            <-- Funciones utilizando DuckDB
| │   │   └── ijson_backend.py             <-- Funciones utilizando ijson
| │   ├── q1_time.py                       <-- q1 optimizada en tiempo
| │   ├── q1_memory.py                     <-- q1 optimizada en memoria
| │   ├── q2_time.py                       <-- q2 optimizada en tiempo
| │   ├── q2_memory.py                     <-- q2 optimizada en memoria
| │   ├── q3_time.py                       <-- q3 optimizada en tiempo
| │   ├── q3_memory.py                     <-- q3 optimizada en memoria
| │   └── benchmark/
| │       ├── benchmark_script.py          <-- Script independiente de benchmarking
| │       └── benchmark.ipynb              <-- Notebook de benchmarking detallado
| ├── profile_reports/
| │   ├── cprofile_combined.txt            <-- Reporte único de cProfile
| │   └── master_summary.csv               <-- CSV con todas las métricas
| ├── run_all_benchmarks.sh                <-- Script maestro para invocar el benchmarking
| ├── requirements.txt                     <-- Versiones de librerías usadas
| ├── tests/
| │   ├── conftest.py                      <-- Fixture de pytest que parametriza backends
| │   ├── data_fixtures.py                 <-- Creación de archivos NDJSON temporales
| │   ├── loaders/
| │   │   ├── test_duckdb_loader.py        <-- Tests de helpers de DuckDB
| │   │   ├── test_pandas_loader.py        <-- Tests de helpers de Pandas
| │   │   └── test_polars_loader.py        <-- Tests de helpers de Polars
| │   └── business/
| │       ├── test_top_active_dates.py     <-- Tests de negocio para top_active_dates
| │       ├── test_top_emojis.py           <-- Tests de negocio para top_emojis
| │       └── test_top_mentioned_users.py  <-- Tests de negocio para top_mentioned_users
| └── README.md                            <-- Descripción general y pasos para ejecutar
```

---

In [2]:
# Ruta del archivo
data_path = Path("../data/farmers-protest-tweets-2021-2-4.json")

## 2. Enfoque general

1. **Separación de funciones y backends**  
   Cada pregunta (q1, q2 y q3) se resuelve con dos funciones:
   - Una versión **optimizada en tiempo** (`qX_time.py`), usando polars.
   - Una versión **optimizada en memoria** (`qX_memory.py`), usando `ijson`.

2. **Backends modulares en `src/backend/`**  
   - `pandas_backend.py`  
   - `polars_backend.py`  
   - `duckdb_backend.py`  
   - `ijson_backend.py`  
   Cada uno expone funciones genéricas (`top_active_dates`, `top_emojis`, `top_mentioned_users`) que se emplean para medir rendimiento.

3. **Benchmark independiente**  
   - El archivo `src/benchmark/benchmark_script.py` ejecuta cada combinación `backend + función` en un **proceso aislado**.  
   - El script genera un CSV acumulativo (`profile_reports/master_summary.csv`) y opcionalmente un único archivo de texto con todos los reportes de `cProfile`.  
   - El script maestro `run_all_benchmarks.sh` lanza las 12 pruebas (4 backends × 3 funciones).

4. **Notebook de benchmark**  
   Para más detalles del benchmark, detalles del optimo de memoria/tiempo y como se escogión la opción óptima para cada caso, ir al notebook correspondiente al benchmarking.
   - `src/benchmark/benchmark.ipynb` importa `profile_reports/master_summary.csv` y genera gráficos comparativos por función (tiempo y memoria).  
  A continuación podemos ver un pequeño cuadro resumen del benchmark:

| Función               | Backend | Tiempo (s) | Peak de memoria (MB) |
|-----------------------|---------|------------|-------------------|
| **top_active_dates**  | pandas  | 1.977      | 515.30            |
|                       | polars  | 0.753      | 751.41            |
|                       | duckdb  | 0.898      | 1626.23           |
|                       | ijson   | 2.749      | 101.88            |
| **top_emojis**        | pandas  | 1.716      | 187.78            |
|                       | polars  | 0.109      | 549.78            |
|                       | duckdb  | 0.646      | 534.25            |
|                       | ijson   | 1.853      | 90.03             |
| **top_mentioned_users** | pandas  | 1.741      | 274.47            |
|                         | polars  | 0.354      | 1241.73           |
|                         | duckdb  | 0.403      | 847.69            |
|                         | ijson   | 2.920      | 94.48             |

---

## 3. Librerías y herramientas usadas

- **cProfile** + **pstats**: para medición detallada de tiempo de CPU, número de llamadas y estadísticas acumuladas.  
- **memory-profiler**: para muestreo y captura del peak de memoria residente durante la ejecución.  
- **ijson**: para lecturas en streaming y minimización de uso de memoria en las funciones optimizadas en memoria.  
- **Pandas**, **Polars**, **DuckDB**: diferentes enfoques en tiempo con estructuras tabulares/columnar/SQL embebido.  
- **Matplotlib**: para graficar comparaciones de tiempo y memoria en el notebook de benchmarking.  
- **Argparse**: en `benchmark_script.py` para recibir parámetros de línea de comandos.

---

## 4. Solución detallada de cada pregunta

### 4.1. Q1: Top 10 fechas con más tweets (+ usuario que más tuitea)

- **Versión optimizada en tiempo**  
  Se usó **Polars** para:
  1. `scan_ndjson()` para leer únicamente las columnas `date` y `user`.
  2. Convertir el campo de fecha a tipo `Datetime` y luego extraer la columna `dt` (solo fecha).
  3. Extraer `user.username` en forma de columna.
  4. Agrupar por `(dt, username)` y contar en paralelo.
  5. Obtener el usuario con mayor count por fecha (`group_by("dt").first()` tras ordenar).
  6. Calcular el total de tweets por `dt` y ordenar para elegir las 10 fechas con más tweets.

- **Versión optimizada en memoria**  
  Se usó **ijson** para:
  1. Abrir el archivo en modo streaming y recorrer cada tweet uno a uno.
  2. Para cada tweet, extraer la fecha (`fecha = date.fromisoformat(date_str[:10])`) y `username`.
  3. Incrementar un `Counter` global para contar tweets totales por fecha.
  4. Mantener un diccionario de `Counter` por fecha que acumula conteo de tuits por usuario.
  5. Al finalizar la lectura, calcular manualmente las 10 fechas con mayor total y extraer el usuario más frecuente en cada fecha.

---

In [3]:
# Q1 optimizada en memoria
from q1_memory import q1_memory

q1_memory_result = q1_memory(data_path)
q1_memory_result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [4]:
# Q1 optimizada en tiempo
from q1_time import q1_time

q1_time_result = q1_time(data_path)
q1_time_result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

### 4.2. Q2: Top 10 emojis más usados (con su conteo)

- **Versión optimizada en tiempo**  
  - Se usó **Polars** para:  
    1. `scan_ndjson()` lee únicamente la columna `content` en modo lazy.  
    2. Filtra tweets con contenido no nulo.  
    3. Crea una expresión que extrae todos los emojis usando una expresión regular Unicode (`\p{Extended_Pictographic}`) en forma vectorizada.  
    4. Aplana la lista de emojis, agrupa por cada símbolo y cuenta en paralelo.  
    5. Ordena por frecuencia y toma los 10 emojis más frecuentes.  

- **Versión optimizada en memoria**  
  - Se usó **ijson** para:  
    1. Abrir el archivo NDJSON en modo streaming y procesar tweet a tweet.  
    2. Para cada tweet, extraer el campo `content` y aplicar un patrón `regex` Python `r"\p{Extended_Pictographic}"` para extraer emojis.  
    3. Mantener un `Counter` que acumula la frecuencia de cada emoji encontrado.  
    4. Al terminar, devolver los 10 emojis más comunes.  

---

In [5]:
# Q2 optimizada en memoria
from q2_memory import q2_memory

q2_memory_result = q2_memory(data_path)
q2_memory_result

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

In [6]:
# Q2 optimizada en tiempo
from q2_time import q2_time

q2_time_result = q2_time(data_path)
q2_time_result

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

### 4.3. Q3: Top 10 usuarios más mencionados (conteo de `@username`)

- **Versión optimizada en tiempo**  
  - Se usó **Polars** para:  
    1. `scan_ndjson()` lee únicamente la columna `mentionedUsers` en modo lazy.  
    2. Explota la columna de listas de menciones (`explode("mentionedUsers")`).  
    3. Mapea cada elemento (un objeto JSON) a su campo `username`.  
    4. Filtra nulos, agrupa por cada `username` y cuenta en paralelo.  
    5. Ordena por frecuencia y toma los 10 usuarios más mencionados.  

- **Versión optimizada en memoria**  
  - Se usa **ijson** para:  
    1. Leer el NDJSON en streaming, tweet a tweet.  
    2. Para cada tweet, extraer la lista `mentionedUsers`.  
    3. Iterar esa lista y, por cada diccionario de mención, extraer `username` y actualizar un `Counter`.  
    4. Al finalizar la lectura, retornar los 10 usuarios más frecuentes del `Counter`.  

---

In [7]:
# Q3 optimizada en memoria
from q3_memory import q3_memory

q3_memory_result = q3_memory(data_path)
q3_memory_result

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

In [8]:
# Q3 optimizada en tiempo
from q3_time import q3_time

q3_time_result = q3_time(data_path)
q3_time_result

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

## 5. Unit Testing

Para garantizar la correcta funcionalidad —y la consistencia entre backends— se implementó una batería completa de tests con **pytest**, organizada en `tests/`.

### 5.1. Estrategia

1. **Parametrización por backend**  
   - Con `conftest.py` se importa dinámicamente cada módulo (`pandas_backend`, `polars_backend`, `duckdb_backend`, `ijson_backend`).  
   - Cada test de negocio (`top_active_dates`, `top_emojis`, `top_mentioned_users`) se ejecuta **cuatro veces**, una por backend.

2. **Fixtures auto-contenidas**  
   - `tests/data_fixtures.py` crea archivos NDJSON temporales usando `tmp_path`, de modo que los tests no dependen de archivos reales ni de conexión externa.  
   - Se cubren escenarios:
     - Datos “felices” (`sample_data_path`)  
     - Casos con valores nulos (`sample_data_with_nulls`)  
     - Dataset sin emojis (`sample_data_no_emoji`)

3. **Cobertura de helpers**  
   - Se testean también las funciones auxiliares internas (`_lazy_scan`, `_load_ndjson`, `_get_connection`, etc.), verificando manejo de rutas inexistentes y retorno de estructuras correctas.

### 5.2. Resultados de cobertura

| Name                            | Stmts | Miss | Cover |
|---------------------------------|-------|------|-------|
| src/backend/__init__.py         | 0     | 0    | 100%  |
| src/backend/duckdb_backend.py   | 59    | 3    | 95%   |
| src/backend/ijson_backend.py    | 92    | 14   | 85%   |
| src/backend/pandas_backend.py   | 56    | 3    | 95%   |
| src/backend/polars_backend.py   | 39    | 3    | 92%   |
| **TOTAL**                       | 246   | 23   | 91%   |

- **Cobertura global:** **91 %** (44 tests, ~0.6 s).  
- **Rango por archivo:** 85 % – 95 %.

### 5.3. Deuda técnica

Aunque un 91 % es aceptable, mi objetivo habitual es más de 95 %.  
Por limitaciones de tiempo dejé pendiente:

| Archivo | Pendiente | Impacto |
|---------|-----------|---------|
| `ijson_backend.py` | Verificar rutas de error en `_parse_ndjson_stream` y branches que manejan objetos JSON anidados poco comunes. | +3/4 % cobertura |
| `duckdb_backend.py` | Tests sobre manejo de tweets sin campo `mentionedUsers` y fechas malformadas. | +1/2 % cobertura |
| Casos borde globales | Archivos vacíos, líneas en blanco múltiples, mezclas de codificaciones. | Robustece producción |

Esta mejora queda como **deuda técnica**.

## 6. Conclusiones finales y elección de frameworks

1. **Resultados clave**  
   - **Polars** fue el backend más rápido en las tres funciones.  
   - **ijson** alcanzó siempre el menor peak de memoria.  
   - **Pandas** ofreció un equilibrio razonable (tiempo medio, memoria contenida).  
   - **DuckDB** igualó a Polars en tiempo en algunos casos, pero con mayor uso de RAM.

2. **Supuesto: ejecución local**  
   Aunque el desafío permite usar plataformas cloud, decidí perfilar en modo single-node por:
   - **Volumen**: 400 MB caben cómodamente en la RAM de un portátil; no se requiere cluster.  
   - **Reproducibilidad rápida**: quien revise el repo puede correr todo en su máquina sin credenciales externas.  
   - **Plazo corto** (4 días): evita sobrecarga de aprovisionar servicios.

   > *Este es un supuesto mío; no es una exigencia del enunciado.*

3. **Cuándo migrar a BigQuery / Dataflow / DataProc**  
   | Escenario | Ventaja cloud-scale |
   |-----------|--------------------|
   | Datasets de **10 GB – TB** | Almacén columnar distribuido (BigQuery) o Spark (DataProc) para escaneo paralelo. |
   | **ETL recurrente** | Apache Beam (runner Dataflow) gestiona ventanas, reintentos y autoscaling. |
   | **Consultas ad-hoc multi-usuario** | BigQuery ofrece capacidad elástica sin afectar a otros workloads. |
   | **SLA estrictos** o alta concurrencia | Delegar tolerancia a fallos y escalado horizontal al proveedor. |

4. **Conclusión práctica**  
   - Para el dataset del reto, **Polars + ijson** cubren los extremos de velocidad y frugalidad en RAM con complejidad mínima.  
   - En proyectos productivos con fuentes continuas o volúmenes muy superiores, escalar a servicios cloud sería lo apropiado; las funciones aquí implementadas podrían portarse a PySpark, Beam o consultas SQL en BigQuery sin cambiar la lógica de negocio.



## A. Anexo - Ejecución de todos los backends

#### Polars

In [9]:
from backend.polars_backend import (
    top_active_dates,
    top_emojis,
    top_mentioned_users
)

# Top Active Dates
n_dates = 10

active_dates = top_active_dates(data_path, n=n_dates)

print(f"**Top {n_dates} dates with the most tweets and their most active user:**\n")
for idx, (date, user) in enumerate(active_dates, start=1):
    print(f"{idx:2d}. Date: {date}   —   User with the most tweets: {user}")

# Top Emojis
n_emojis = 10

most_used_emojis = top_emojis(data_path, n=n_emojis)

print(f"\n**Top {n_emojis} most used emojis (emoji — frequency):**\n")
for idx, (emoji, frequency) in enumerate(most_used_emojis, start=1):
    print(f"{idx:2d}. {emoji}   —   {frequency}")

# Top Mentions
n_mentions = 10

most_mentioned_users = top_mentioned_users(data_path, n=n_mentions)

print(f"\n**Top {n_mentions} most mentioned users (user — frequency):**\n")
for idx, (user, count) in enumerate(most_mentioned_users, start=1):
    print(f"{idx:2d}. {user}   —   {count}")

**Top 10 dates with the most tweets and their most active user:**

 1. Date: 2021-02-12   —   User with the most tweets: RanbirS00614606
 2. Date: 2021-02-13   —   User with the most tweets: MaanDee08215437
 3. Date: 2021-02-17   —   User with the most tweets: RaaJVinderkaur
 4. Date: 2021-02-16   —   User with the most tweets: jot__b
 5. Date: 2021-02-14   —   User with the most tweets: rebelpacifist
 6. Date: 2021-02-18   —   User with the most tweets: neetuanjle_nitu
 7. Date: 2021-02-15   —   User with the most tweets: jot__b
 8. Date: 2021-02-20   —   User with the most tweets: MangalJ23056160
 9. Date: 2021-02-23   —   User with the most tweets: Surrypuria
10. Date: 2021-02-19   —   User with the most tweets: Preetm91

**Top 10 most used emojis (emoji — frequency):**

 1. 🙏   —   7286
 2. 😂   —   3072
 3. 🚜   —   2972
 4. ✊   —   2411
 5. 🌾   —   2363
 6. ❤   —   1779
 7. 🤣   —   1668
 8. 👇   —   1108
 9. 💚   —   1040
10. 💪   —   947

**Top 10 most mentioned users (user — frequen

#### Pandas

In [10]:
from backend.pandas_backend import (
    top_active_dates,
    top_emojis,
    top_mentioned_users
)

# Top Active Dates
n_dates = 10

active_dates = top_active_dates(data_path, n=n_dates)

print(f"**Top {n_dates} dates with the most tweets and their most active user:**\n")
for idx, (date, user) in enumerate(active_dates, start=1):
    print(f"{idx:2d}. Date: {date}   —   User with the most tweets: {user}")

# Top Emojis
n_emojis = 10

most_used_emojis = top_emojis(data_path, n=n_emojis)

print(f"\n**Top {n_emojis} most used emojis (emoji — frequency):**\n")
for idx, (emoji, frequency) in enumerate(most_used_emojis, start=1):
    print(f"{idx:2d}. {emoji}   —   {frequency}")

# Top Mentions
n_mentions = 10

most_mentioned_users = top_mentioned_users(data_path, n=n_mentions)

print(f"\n**Top {n_mentions} most mentioned users (user — frequency):**\n")
for idx, (user, count) in enumerate(most_mentioned_users, start=1):
    print(f"{idx:2d}. {user}   —   {count}")

**Top 10 dates with the most tweets and their most active user:**

 1. Date: 2021-02-12   —   User with the most tweets: RanbirS00614606
 2. Date: 2021-02-13   —   User with the most tweets: MaanDee08215437
 3. Date: 2021-02-17   —   User with the most tweets: RaaJVinderkaur
 4. Date: 2021-02-16   —   User with the most tweets: jot__b
 5. Date: 2021-02-14   —   User with the most tweets: rebelpacifist
 6. Date: 2021-02-18   —   User with the most tweets: neetuanjle_nitu
 7. Date: 2021-02-15   —   User with the most tweets: jot__b
 8. Date: 2021-02-20   —   User with the most tweets: MangalJ23056160
 9. Date: 2021-02-23   —   User with the most tweets: Surrypuria
10. Date: 2021-02-19   —   User with the most tweets: Preetm91

**Top 10 most used emojis (emoji — frequency):**

 1. 🙏   —   7286
 2. 😂   —   3072
 3. 🚜   —   2972
 4. ✊   —   2411
 5. 🌾   —   2363
 6. ❤   —   1779
 7. 🤣   —   1668
 8. 👇   —   1108
 9. 💚   —   1040
10. 💪   —   947

**Top 10 most mentioned users (user — frequen

#### DuckDB

In [11]:
from backend.duckdb_backend import (
    top_active_dates,
    top_emojis,
    top_mentioned_users
)

# Top Active Dates
n_dates = 10

active_dates = top_active_dates(data_path, n=n_dates)

print(f"**Top {n_dates} dates with the most tweets and their most active user:**\n")
for idx, (date, user) in enumerate(active_dates, start=1):
    print(f"{idx:2d}. Date: {date}   —   User with the most tweets: {user}")

# Top Emojis
n_emojis = 10

most_used_emojis = top_emojis(data_path, n=n_emojis)

print(f"\n**Top {n_emojis} most used emojis (emoji — frequency):**\n")
for idx, (emoji, frequency) in enumerate(most_used_emojis, start=1):
    print(f"{idx:2d}. {emoji}   —   {frequency}")

# Top Mentions
n_mentions = 10

most_mentioned_users = top_mentioned_users(data_path, n=n_mentions)

print(f"\n**Top {n_mentions} most mentioned users (user — frequency):**\n")
for idx, (user, count) in enumerate(most_mentioned_users, start=1):
    print(f"{idx:2d}. {user}   —   {count}")

**Top 10 dates with the most tweets and their most active user:**

 1. Date: 2021-02-12   —   User with the most tweets: RanbirS00614606
 2. Date: 2021-02-13   —   User with the most tweets: MaanDee08215437
 3. Date: 2021-02-17   —   User with the most tweets: RaaJVinderkaur
 4. Date: 2021-02-16   —   User with the most tweets: jot__b
 5. Date: 2021-02-14   —   User with the most tweets: rebelpacifist
 6. Date: 2021-02-18   —   User with the most tweets: neetuanjle_nitu
 7. Date: 2021-02-15   —   User with the most tweets: jot__b
 8. Date: 2021-02-20   —   User with the most tweets: MangalJ23056160
 9. Date: 2021-02-23   —   User with the most tweets: Surrypuria
10. Date: 2021-02-19   —   User with the most tweets: Preetm91

**Top 10 most used emojis (emoji — frequency):**

 1. 🙏   —   7286
 2. 😂   —   3072
 3. 🚜   —   2972
 4. ✊   —   2411
 5. 🌾   —   2363
 6. ❤   —   1779
 7. 🤣   —   1668
 8. 👇   —   1108
 9. 💚   —   1040
10. 💪   —   947

**Top 10 most mentioned users (user — frequen

#### ijson

In [12]:
from backend.ijson_backend import (
    top_active_dates,
    top_emojis,
    top_mentioned_users
)

# Top Active Dates
n_dates = 10

active_dates = top_active_dates(data_path, n=n_dates)

print(f"**Top {n_dates} dates with the most tweets and their most active user:**\n")
for idx, (date, user) in enumerate(active_dates, start=1):
    print(f"{idx:2d}. Date: {date}   —   User with the most tweets: {user}")

# Top Emojis
n_emojis = 10

most_used_emojis = top_emojis(data_path, n=n_emojis)

print(f"\n**Top {n_emojis} most used emojis (emoji — frequency):**\n")
for idx, (emoji, frequency) in enumerate(most_used_emojis, start=1):
    print(f"{idx:2d}. {emoji}   —   {frequency}")

# Top Mentions
n_mentions = 10

most_mentioned_users = top_mentioned_users(data_path, n=n_mentions)

print(f"\n**Top {n_mentions} most mentioned users (user — frequency):**\n")
for idx, (user, count) in enumerate(most_mentioned_users, start=1):
    print(f"{idx:2d}. {user}   —   {count}")

**Top 10 dates with the most tweets and their most active user:**

 1. Date: 2021-02-12   —   User with the most tweets: RanbirS00614606
 2. Date: 2021-02-13   —   User with the most tweets: MaanDee08215437
 3. Date: 2021-02-17   —   User with the most tweets: RaaJVinderkaur
 4. Date: 2021-02-16   —   User with the most tweets: jot__b
 5. Date: 2021-02-14   —   User with the most tweets: rebelpacifist
 6. Date: 2021-02-18   —   User with the most tweets: neetuanjle_nitu
 7. Date: 2021-02-15   —   User with the most tweets: jot__b
 8. Date: 2021-02-20   —   User with the most tweets: MangalJ23056160
 9. Date: 2021-02-23   —   User with the most tweets: Surrypuria
10. Date: 2021-02-19   —   User with the most tweets: Preetm91

**Top 10 most used emojis (emoji — frequency):**

 1. 🙏   —   7286
 2. 😂   —   3072
 3. 🚜   —   2972
 4. ✊   —   2411
 5. 🌾   —   2363
 6. ❤   —   1779
 7. 🤣   —   1668
 8. 👇   —   1108
 9. 💚   —   1040
10. 💪   —   947

**Top 10 most mentioned users (user — frequen